In [ ]:
#準備工作
# AllDocRoot 為所有評論所在資料夾, json檔
AllDocRoot = r"C:\Users\User\Desktop\商管\Final\TenReviews"
#CP值的csv轉成utf-8 的txt所在位置, github上我放的那幾個
CPPath = r'C:\Users\User\Documents\GitHub\2019fall_final\CP.txt'
EnvironmentPath = r'C:\Users\User\Documents\GitHub\2019fall_final\環境.txt'
ServicePath = r'C:\Users\User\Documents\GitHub\2019fall_final\服務.txt'
DelicacyPath = r'C:\Users\User\Documents\GitHub\2019fall_final\美味.txt'
#  務必下載斷詞字典和停用詞字典放到以下位置, github上有
SepDictionaryPath = r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\dict.txt'
StopDicPath = r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\stop.txt'
jieba.set_dictionary(r'C:\Users\User\AppData\Local\Programs\Python\Python37\Lib\jieba\dict.txt')


In [125]:
import json
import glob
import os
import jieba
import re

with open(StopDicPath,'r',encoding = 'utf8') as f:
    StopWords = f.read().split('\n')
    StopWords[0] = ','
    
def ChiFeatureSelection(Corpus_list,dic_list,Class_group, n ):
    '''
    Feautures Selection的方法
    Input:Corpus_list, [0] 為標記 0,1 [1]為斷詞後評論
          dic_list, set of terms
          Class_group, 共幾組，以list表示 i.e. ['0','1']
          n, 前幾名term
    '''
    goodfeatures = []
    for term in dic_list:
        Xchi = 0
        ContingencyTable = []
        Tcount= 0
        Fcount = 0
        for Class in Class_group:
            TP = 0
            FN = 0
            for corpus in Corpus_list:
                
                if term in corpus[1] and Class == corpus[0]:
                    TP += 1
                elif term not in corpus[1] and Class == corpus[0]:
                    FN += 1
            Tcount += TP
            Fcount += FN
#             print(Tcount)
            ContingencyTable.append([TP,FN])
#             print(Class)
            
        Total = (Tcount + Fcount) 
        T_P = Tcount / Total
        N_P = 1 - T_P
#         print(T_P,N_P,sep=' ')
        for i in ContingencyTable:
            T = i[0]
            F = i[1]
            Class_P = (T + F)/ Total
            
            Etc = T_P * Class_P
            Efc = N_P * Class_P
            if N_P ==0:
                print(N_P)
            Xchi += (T - Etc)**(2) / Etc + (F - Efc)**(2) / Efc
        if ContingencyTable[0][0] > ContingencyTable[1][0]:
            Postive = '0'
        elif ContingencyTable[0][0] < ContingencyTable[1][0]:
            Postive = '1'
        else:
            Postive = 'equal'
        goodfeatures.append([Postive,term,Xchi])
    goodfeatures.sort(reverse = True, key = lambda x:x[2])
    goodfeatures_term = []
    for i in goodfeatures:
        goodfeatures_term.append([i[0],i[1],i[2]])

    return goodfeatures_term[0:n]

# def DocsSepbyVendor(ALLDocsRoot):
#     import json
#     import requests
#     import glob
#     import os
        
#     file_list = glob.glob(os.path.join(os.getcwd(), ALLDocsRoot, "*.json"))
#     CorpusByVendor = dict()
        
#     for file_path in file_list:
#         BaseName = file_path.split('\\')[-1].strip('.json')
#         with open(file_path,encoding = 'utf-8') as load_test:
#             doc =json.load(load_test)

#             CorpusPerVendorTemp= []
    
#             if len(doc['reviews']) != 0:
#                 for i in range(len(doc['reviews'])):
#                     Text = doc['reviews'][i]['text']
#                     Text = re.findall(r'[\u4e00-\u9fff]+', Text)
#                     Sisstring = ''
#                     for i in Text:
#                         Sisstring += i 
#                     words = [s for s in jieba.cut(Sisstring, cut_all=False) if s not in StopWords]
#                     CorpusPerVendorTemp.extend(words)
#                 CorpusByVendor[BaseName] = CorpusPerVendorTemp
#             else:
#                  pass
#     return CorpusByVendor 

def Tf_idf(terms , CorpusByVendor,ChiDict):
    dic = {} 

    for item in terms: 
        if (item not in dic): 
            dic[item] = 0

    for keys in dic.keys():

        for vendor in CorpusByVendor.keys():

            if (keys in CorpusByVendor[vendor]):
                dic[keys] += 1

            else:
                dic[keys] += 0
    dicSorted = sorted(dic.items() ,  key=lambda x: x[1])
#     print(dicSorted)
    import math
    Rank = []
    for keys in CorpusByVendor.keys() :
        
        docfreq = []
        docunit = []
        N =len(CorpusByVendor.keys())
        if len(CorpusByVendor[keys]) != 0:
#             print(CorpusByVendor[keys])
            for f in dicSorted:
                if (f[0] in CorpusByVendor[keys]):
                    tf = CorpusByVendor[keys].count(f[0])
    #                 N =len(file_list)
                    df = dic[f[0]]
                    idf = math.log((N / df) ,10)
                    tf_idf = tf * idf
                else:
                    tf_idf = 0
                
                docfreq.append([tf_idf,ChiDict[f[0]]])

            sum_square = 0
            for i in docfreq:
                sum_square += i[0]**2
            if sum_square != 0:
                for i in docfreq:
                    docunit.append([i[0]/(sum_square**0.5),i[1]])
                Score = 0
                for i in docunit:
                    Score += i[0] * i[1]
            else:
                    Score = 0
            Rank.append([keys,Score])    
    Rank.sort(reverse = True,key = lambda x: x[1])
    return Rank

In [138]:
class Training:
    import jieba
    
    def __init__(self,Path):
        '''
        Input: Path, 為訓練集的位置的，UTF-8 txt
        Output: Positive, 正面(1)或負面(0)
                Name, 店家名
                Counter ,該店家第幾篇評論
        '''
        Corpus = []
        with open(Path,'r', encoding = 'utf-8') as text:
        #     text.close()
            PositiveList = []
            NameList = []
            CounterList = []
            for i in text:
                Positive = i.strip('\n').split('\t')[0]
                Positive = Positive.strip('\ufeff')
                Name = i.strip('\n').split('\t')[1].split(',')[0]
                Id =int(i.strip('\n').split('\t')[1].split(',')[1])
                PositiveList.append(Positive)
                NameList.append(Name)
                CounterList.append(Id)
                
        self.Positive = PositiveList
        self.Name= NameList
        self.Counter = CounterList
        
    def BuildTrainDic(self,ALLDocsRoot,n):
        '''
        Input:  ALLDocsRoot, 為所有評論的位置，評論為json檔
                Positive, 正面(1)或負面(0)
                Name, 店家名
                Counter ,該店家第幾篇評論
                皆為上述產生
                n, 前n名的Feautures
        Ootput:
                self.TopNFeautures,前n名的Feautures
                
        41~70 對你找出資料夾中訓練集的文章並清掉韓文有幫助，71是結疤斷詞
        '''
        file_list = glob.glob(os.path.join(os.getcwd(), ALLDocsRoot, "*.json"))
        Corpus = []
        NegativeCorpus = []
        NameAndPositive = []
        Potential_Features = []
        Potential_GFeatures = []
        Potential_NFeatures = []
        for i in zip(self.Positive,self.Name):
            NameAndPositive.append([i[1],i[0]])
       
        for file_path in file_list:
            BaseName = file_path.split('\\')[-1].strip('.json')
            for NamePositiveAndOrder in zip(NameAndPositive,self.Counter):
#                 print(NamePositiveAndOrder[0][0],BaseName)
                if BaseName == NamePositiveAndOrder[0][0] :                   
                    with open(file_path,encoding = 'utf-8') as load_test:
                        doc =json.load(load_test)
                        
                        if len(doc['reviews']) != 0:
                            
                            for i in range(len(doc['reviews'])):
                                if i == NamePositiveAndOrder[1]:
                                    
                                    Text = doc['reviews'][i]['text']
                                    Text = re.findall(r'[\u4e00-\u9fff]+', Text)
                                    Sisstring = ''
                                    for i in Text:
                                        Sisstring += i 
                                    words = [s for s in jieba.cut(Sisstring, cut_all=False) if s not in StopWords]
                                    if NamePositiveAndOrder[0][1] == '1':
                                        Corpus.append([NamePositiveAndOrder[0][1],words])
#                                         Potential_GFeatures.extend(words)
                                        Potential_Features.extend(words)
                                       
                                    elif NamePositiveAndOrder[0][1] == '0':
                                        Corpus.append([NamePositiveAndOrder[0][1],words])
#                                         Potential_NFeatures.extend(words)
                                        Potential_Features.extend(words)
                                    else:
                                        return print('Indicatior is neither 1 or 0')

        self.Potential_Features = set(Potential_Features)
#         self.GoodFeatures = set(Potential_GFeatures)
#         self.NegativeFeatures = set(Potential_NFeatures)
        
        Class_group= ['0','1']
        self.TopNFeatures = ChiFeatureSelection(Corpus, self.Potential_Features,Class_group, n)
        self.TopNGood = [G[1] for G in self.TopNFeatures if G[0] == '1']
        self.TopNNegative = [G[1] for G in self.TopNFeatures if G[0] == '0']
        self.TopNFeatures = [G[1] for G in ChiFeatureSelection(Corpus, self.Potential_Features,Class_group, n)]
        self.ChiDict ={t[1]:t[2] for t in ChiFeatureSelection(Corpus, self.Potential_Features,Class_group, n)}

class Grading:
    def __init__(self,ALLDocsRoot):
        import json
        import requests
        import glob
        import os

        file_list = glob.glob(os.path.join(os.getcwd(), ALLDocsRoot, "*.json"))
        CorpusByVendor = dict()

        for file_path in file_list:
            BaseName = file_path.split('\\')[-1].strip('.json')
            with open(file_path,encoding = 'utf-8') as load_test:
                doc =json.load(load_test)

                CorpusPerVendorTemp= []

                if len(doc['reviews']) != 0:
                    for i in range(len(doc['reviews'])):
                        Text = doc['reviews'][i]['text']
                        Text = re.findall(r'[\u4e00-\u9fff]+', Text)
                        Sisstring = ''
                        for i in Text:
                            Sisstring += i 
                        words = [s for s in jieba.cut(Sisstring, cut_all=False) if s not in StopWords]
                        CorpusPerVendorTemp.extend(words)
                    CorpusByVendor[BaseName] = CorpusPerVendorTemp
                else:
                     pass
        self.CorpusByVendor = CorpusByVendor
        
    def WeightedScore(self,Training):     
        SepCorpus = self.CorpusByVendor
        
        GoodScore = {t[0]:t[1] for t in Tf_idf(Training.TopNGood,SepCorpus,Training.ChiDict)}
        BadScore = {t[0]:t[1] for t in Tf_idf(Training.TopNNegative,SepCorpus,Training.ChiDict)}

        WeightedScore = {}

        for key in GoodScore.keys():
            WeightedScore[key] = GoodScore[key] - BadScore[key]
        SortedWeightedScore = {k: v for k, v in sorted(WeightedScore.items(),reverse = True, key=lambda item: item[1])}
        self.SortedWeightedScore = SortedWeightedScore



In [142]:
#建立CP字典
AllDocRoot = r"C:\Users\User\Desktop\商管\Final\TenReviews" #所有評論地點
CPPath = r'C:\Users\User\Documents\GitHub\2019fall_final\CP.txt' #確認訓練集標號文件地點

CPTrain = Training(CPPath) #初始化訓練文件，

Training.BuildTrainDic(CPTrain, AllDocRoot, 100) #建立下述物件 而100為找出前100個有判別力的斷詞(不分好壞)

# print(CPTrain.TopNFeatures) # 回傳100判別力的斷詞(不分好壞)
# print(CPTrain.TopNGood) # 回傳100個中有判斷力的正面斷詞
# print(CPTrain.TopNNegative) # 回傳100個中有判斷力負面斷詞 前兩者數量相加應等於TopNFeatures
# print(CPTrain.ChiDict) # 100個term的卡方值字典 待會計算wighting會用到

CPFinal = Grading(AllDocRoot) #初始化排名字典
Grading.WeightedScore(CPFinal, CPTrain) # 建立排名字典 數字越大表示 其正負相抵後的在該項正面指標越高
# print(CPFinal.SortedWeightedScore) #字典結果 

{'Ooh Cha Cha 自然食科技大樓 _ Hooch11-20': 13676.894768292083, 'Barkers11-20': 10561.096261568719, '蕭家傳統小吃11-20': 10096.494903504652, 'Pasta_202_20Go11-20': 9709.301034403514, '川味辛亥小吃11-20': 9298.708925450816, '小飯館兒11-20': 8430.912332972479, 'Riad_20Chi_20Chi異國料理11-20': 8284.074405984986, '松田日式飯糰11-20': 8282.643704110655, '順園小館11-20': 8057.971749058761, '韓天閣11-20': 7777.169120845421, '柒食貳11-20': 7582.097321067908, '鳳城臘味家_20(百合店)11-20': 7193.275225208072, '淺草日式飯糰11-20': 6725.5205265674485, '三米三小館11-20': 6692.357348245119, '大聲公 台大牛肉麵店11-20': 6544.103554729749, '小六手工拉麵11-20': 6493.352179020616, '香港亨記(公館店)11-20': 5995.246705289381, '巧味快餐11-20': 5880.509008961231, '伊斯坦堡土耳其餐酒館11-20': 5778.922272440188, '瑪德蓮小酒館11-20': 5748.434821910978, 'Sababa_20Pita_20Bar沙巴巴中東美食11-20': 5576.573707310175, '牛洞食堂11-20': 5446.7545920282755, 'Mr._20拉麵11-20': 5439.680724574893, '玉欣珍傳統美食坊11-20': 5415.042938589268, '大家素食11-20': 5396.540095360027, '鑫吉野烤肉飯公館店11-20': 5389.674510158315, '憶馬當鮮 馬來西亞風味料理11-20': 5340.78864662592